# Data Analytics with pandas

To begin with, we will concentrate on pandas, as it has the most sophisticated API. Both, cuDF and Dask replicate pandas' API, but do not incorporate all functionalities and methods. We will go through Dask and cuDF in subsequent notebooks.
One of the main take-away messeges of this part of the course should be:
Whenever your data set fits in memory and operations on it execute in an acceptable time frame, you should stick with pandas. When you want to leverage the advantages of GPU acceleration, cuDF is your best friend and should you need to harness the power of parallel execution on CPUs, go with Dask. However, do not expect everything you are familiar with in pandas to work with cuDF or Dask.

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
# import cudf
import os
import glob

## Data Loading

You can use pandas, cuDF and Dask to load a wide range of file formats (csv, xls, parquet, json, hdf, etc.)

### Single files:

In [ ]:
pdf = pd.read_csv("./data/nycflights/1999.csv") # Reads one csv file and displays it as a dataframe

In [ ]:
pdf.head() # Displays the dataframe with the first 5 entries. If you want to view a differnt number of rows, enter the number into the brackets

In [ ]:
pdf.tail() # Displays the last 5 entries of the dataframe. You can choose the number of rows that should be displayed

In [ ]:
pdf.dtypes # Returns the data type of each column. Objects are bacically strings.

### Multiple files

In [ ]:
filepath = glob.glob("./data/nycflights/*.csv")

To read multiple files into one dataframe, you need to concatenate each one with a for-loop:

In [ ]:
pdf = pd.concat(pd.read_csv(f) for f in filepath)

In [ ]:
pdf.head(3)

In [ ]:
pdf.tail(3)

## Introduction to Data Structures

### Series
The pandas documentation describes a series as such:
>One-dimensional ndarray with axis labels (including time series).
Labels need not be unique but must be a hashable type. The object supports both integer- and label-based indexing and provides a host of methods for performing operations involving the index. Statistical methods from ndarray have been overridden to automatically exclude missing data (currently represented as NaN).

In this coure we will no be focussing on series, but should you need to get to know the basics, you can go through the short introduction by yourself.

In [ ]:
obj = pd.Series([-4, 6, -2, 1]) # This is how you create a pandas series from a Python list
obj

In [ ]:
obj.values

In [ ]:
obj.index  # Just like range(4)

In [ ]:
obj2 = pd.Series([-4, 6, -2, 1], index=['w', 'z', 'y', 'x']) # The index doesn't need to be numeric. Let's change it!
obj2

In [ ]:
obj2.index

In [ ]:
obj2['y'] # Values can be selected with the label....

In [ ]:
obj2[2] # ...or the index

In [ ]:
obj2["w"] = 0 # Values can be re-asigned
obj2[["w", "x", "y", "z"]] # You can pass a Pyhon list with index numbers or labels for calues you want to select

In [ ]:
obj2 # The operation above has changed nothing in our series

In [ ]:
obj2[obj2 > 0] # Just as in NumPy-like operations, you can filter using a boolean array....

In [ ]:
obj2 * 2 # ....or perform scalar multiplication....

In [ ]:
np.exp(obj2) # ....or apply mathematical functions. This will always preserve the index-value link....

In [ ]:
obj2 # ...and does not change the actual series object.

In [ ]:
'y' in obj2 # You can use series instead of a dict.

In [ ]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}
obj3 = pd.Series(sdata) # If you have a Python dict, you can create a series from it by passing the dict
obj3 # The series object automatically uses the dict's keys as indexes

In [ ]:
states = ["California", "Ohio", "Oregon", "Texas"]
obj4 = pd.Series(sdata, index=states) # Let's override the default index or key order
obj4 # Since "California" was not a key in our dict, we have a missing value or NaN (not a number)

In [ ]:
obj4.isnull() # You can use the isnull (or notnull) method to detect missing data

In [ ]:
obj3 + obj4 # You can apply arithmetic operations to series. The result will automatically align by index

In [ ]:
obj4.index = ["Bob", "Steve", "Jeff", "Ryan"] # You can easily rename the index entries
obj4

### DataFrame
The docstring of the DataFrame class describes this object like this:
>Two-dimensional, size-mutable, potentially heterogeneous tabular data.
    Data structure also contains labeled axes (rows and columns).
    Arithmetic operations align on both row and column labels. Can be
    thought of as a dict-like container for Series objects. The primary
    pandas data structure.

In this course we will mainly be focussing on DataFrames, as they are the workhorse in pandas and ilustrate the capabilities of cuDF and Dask really well.

In [ ]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data) # There are many ways of how to construct a DataFrame, but this is one of the most common ones 

In [ ]:
frame

In [ ]:
frame.head()

In [ ]:
pd.DataFrame(data, columns=["year", "state", "pop"]) # Let's re-arrange the columns

In [ ]:
frame2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"], # Here we create a new DataFrame with new columns and and indices
                      index=["one", "two", "three", "four",
                             "five", "six"])
frame2 # The column "debt" does not currently have any values

In [ ]:
frame2.columns 

In [ ]:
frame2["state"] # A column can be retreived as a series either by dict-like notation....

In [ ]:
frame2.state # ....or by attribute

In [ ]:
frame2[["state"]] # Pass a Python list to get a Dataframe instead of a series

In [ ]:
frame2.loc["three"] # To retrieve a row, use the loc attribute

In [ ]:
frame2["debt"] = 16.5 # Columns can be modified by assignment. Here, we assign the empty "debt" column a scalar value
frame2

In [ ]:
frame2["debt"] = np.arange(6.) # Here we assign an array of values. When you assign lists or arrays to a column, the value's length must match the length of the Dataframe.
frame2

In [ ]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five']) # If you assign a series, its labels will be realigned exactly to the DataFrame's index, inserting NaNs in any holes
frame2['debt'] = val
frame2

In [ ]:
frame2["eastern"] = frame2.state == "Ohio" # Assigning a column that doesn't exist creates a new column
frame2

In [ ]:
del frame2["eastern"] # Should you want to delete a column, use the del keyword....
frame2

In [ ]:
frame2.drop(["debt"], axis = 1, inplace = True) # ...or the drop method. Here you have to specify the label and axis. If you permanently want to delete the column/row, set inplace=True
frame2

### Index Objects
panda's index objects are responsible for holding the axis labels and other metadata (like the axis name or names). Any array or other sequence of labels you use when constructing a series or DataFrame is internally converted into an index. Index objects are immutable and thus cant't be modified by the user, making it safer to share index objects among data structures.
In this course we will no discuss index objects, but feel free to go through that section in your own time.

In [ ]:
obj = pd.Series(range(3), index=["a", "b", "c"])
index = obj.index
index

In [ ]:
index[1:]

In [ ]:
labels = pd.Index(np.arange(3))
labels

In [ ]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

In [ ]:
obj2.index is labels

In [ ]:
frame2

In [ ]:
frame2.columns

In [ ]:
"state" in frame2.columns

In [ ]:
"one" in frame2.index

In [ ]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar']) # A pandas index can contain duplicate labels. Selections with duplicate labels will select all occurrences of that label.
dup_labels

## Essential Functionality

### Reindexing and Renaming

Let's create a random DataFrame:

In [ ]:
frame = pd.DataFrame(np.arange(81).reshape((9, 9)),
                     index=["a", "c", "i", "d", "f", "g", "b", "e", "h"],
                     columns=["ba", "be", "bi", "bo", "bu", "ca", "ce", "ci", "co"])
frame

Reindex moves whole rows about (including values), not just the index

In [ ]:
frame2 = frame.reindex(["a", "b", "c", "d", "e", "f", "g", "h", "i"])
frame2

This is a list of Austrian provincial capitals

In [ ]:
cities = ["Vienna", "Salzburg", "Linz", "Graz", "Bregenz", "Innsbruck", "Klagenfurt", "Eisenstadt", "Sankt Pölten"]

With set_axis we can rename columns or indices

In [ ]:
frame2.set_axis(cities, axis=1, inplace=True)
frame2

In [ ]:
cities_new_order = ["Salzburg", "Vienna", "Graz", "Linz", "Bregenz", "Innsbruck", "Klagenfurt", "Eisenstadt", "Sankt Pölten"]

In [ ]:
frame3 = frame2.reindex(columns = cities_new_order) # As before, should we want to change the order of columns, we use reindex.
frame3

Now, let's work on the flight DataFrame from above:

In [ ]:
pdf # Just to remind us how that DataFrame looked like

The indices do not match the number of rows, so let's reset the index:

In [ ]:
pdf2 = pdf.reset_index()

We would like to combine year, month and day into a date column. To do so, we first need to rename the columns, according to the expected keywords in the next command.

In [ ]:
pdf2.rename(columns={"Year":"year", "Month":"month", "DayofMonth":"day"}, inplace=True)
pdf2

In [ ]:
pdf2["date"] = pd.to_datetime(pdf2[["year", "month", "day"]]) # Now, we can combine the three columns into a new date column....
pdf2

In [ ]:
pdf2.dtypes # The date column indeed has the datetime datatype

In [ ]:
pdf2.set_index("date", inplace=True, drop=True) # .....and set the date as index

In [ ]:
pdf2

### Handling Missing Data

We use the notna() method to select all the rows without missing values

In [ ]:
pdf2[pdf2[["year", "month", "day", "DepTime", "ArrTime", "FlightNum", "ArrDelay", "DepDelay", "Origin", "Dest", "Cancelled"]].notna()]

The isna() method does the exact opposite and selects the rows with missing values

In [ ]:
pdf2[pdf2["DepTime"].isna()]

Let's reduce our DataFrame to make it a bit more accessible:

In [ ]:
pdf3 = pdf2[["year", "month", "day", "DepTime", "ArrTime", "FlightNum", "ArrDelay", "DepDelay", "Origin", "Dest", "Cancelled"]]
pdf3.shape # This gives you the number of rows and columns

To select rows with actual values we use the notnull() method. Unfortunately, we cannot pass a list with column labels, but have to repeat the command for each clolumn name. AS the cancelled flights are the ones wich have no departure and arrival times, we automatically select the flights which were not cancelled.

In [ ]:
pdf3[pdf3["Cancelled"]==1].head()

We would like to use the cancellations at a later point, therefore we create a new DataFrame:

In [ ]:
pdf4 = pdf3
pdf4 = pdf4[pdf4["DepTime"].notnull()] # notnull() is equivalent to notna()
pdf4 = pdf4[pdf4["ArrTime"].notnull()]
pdf4 = pdf4[pdf4["ArrDelay"].notnull()]
pdf4 = pdf4[pdf4["DepDelay"].notnull()]
pdf4.shape

### Data Transformation

We would like to transform the departure time and arrival time into a datetime format. However, first we need to transform the float into a correctly rounded integer. For that we use the round() and astype() methods.

In [ ]:
pdf4[["DepTime", "ArrTime", "ArrDelay", "DepDelay"]] = pdf4[["DepTime", "ArrTime", "ArrDelay", "DepDelay"]].round(0).astype(int)
pdf4

To transform the number into a datetime format we first need to transform it into a string:

In [ ]:
pdf4[["DepTime", "ArrTime"]] = pdf4[["DepTime", "ArrTime"]].astype(str)

Some of the strings only have three characters. With this lambda function we make sure that all of them are made up of four characters:

In [ ]:
pdf4["DepTime"] = pdf4["DepTime"].apply(lambda x: x.zfill(4))
pdf4["ArrTime"] = pdf4["ArrTime"].apply(lambda x: x.zfill(4))
pdf4

In [ ]:
pdf4[pdf4["DepTime"].str.len()!=4] # let' just make sure that that all values have four characters

In [ ]:
pdf4.dtypes # The DepTime and ArrTime are indeed strings (objects)

We can finally transform these columns into datetime datatypes. Here we pass exact=False, incase some of the strings do not have the ideal format. The .dt.time at the end makes sure, we only select the time and not the date.

In [ ]:
pdf4["DepTime"] = pd.to_datetime(pdf4["DepTime"], format="%H%M", exact=False).dt.time

In [ ]:
pdf4["ArrTime"] = pd.to_datetime(pdf4["ArrTime"], format="%H%M", exact=False).dt.time

In [ ]:
pdf4

### Dropping Entries from an Axis

The drop() method drops entries from axis 0 (rows) by default:

In [ ]:
frame3.drop(["d", "g"])

You need to pass axis=1 or axis = "columns" to drop columns:

In [ ]:
frame3.drop(["Eisenstadt", "Sankt Pölten"], axis="columns")

Maybe you noticed, that the drop method does not change the original dataframe by default. If you wish to do so, pass inplace=True.

In [ ]:
frame3.drop("Vienna", axis=1, inplace=True)
frame3

How would you drop the "year", "month" and "day" column in the pdf4 DataFrame?

In [ ]:
# Your turn:


Solution:

In [ ]:
pdf4.drop(["year", "month", "day"], axis="columns", inplace=True)
pdf4

### Indexing, Selection, and Filtering

Passing a single element or a list to the [] operator selects columns.

In [ ]:
frame3["Salzburg"]

To select rows use the dataframe[3:7] slicing sytax:

In [ ]:
frame3[0:4]

For DataFrame label-indexing on the rows, use loc:

In [ ]:
frame3.loc[["a","b"]]

With loc you can also select rows and columns at the same time:

In [ ]:
frame3.loc[["a","b"],["Salzburg", "Graz"]]

As we now have a duplicate indices the loc method gives all entries with that date:

In [ ]:
pdf4.loc["1999-01-01"]

Let's select data with a boolean array:

In [ ]:
frame3 < 10

In [ ]:
frame3[frame3["Salzburg"] < 20]

Here we set the values in column "Salzburg" which are below 20 equal to zero:

In [ ]:
frame3["Salzburg"][frame3["Salzburg"] < 20] = 0
frame3

### Arithmetic and Data Alignment

In arithmetic operations between DataFrames, the internal data alignment indtroduces missing values in the label locations (indices and columns) that don't overlap.

In [ ]:
frame4 = pd.DataFrame(np.arange(36).reshape((6, 6)),
        columns = ["Vienna", "Salzburg", "Graz", "Bregenz", "Innsbruck", "Eisenstadt"],
        index = ["a", "c", "d", "f", "g", "i"])

Adding these together returns a DataFrame whose index and columns are the unions of the ones in frame3 and frame4:

In [ ]:
frame3 + frame4

Use arithmetic methods, if you want to fill the not overlapping areas with specific values:

In [ ]:
frame3.add(frame4, fill_value=0) # Now only areas where data is missing in both Dataframes have NaN entries.

### Function Application and Mapping

NumPy ufuncs (element-wise array methods) also work with pandas objects:

In [ ]:
np.square(frame3)

Another frequent operation is applying a function on one-dimensional arrays to each column or row. Here the function f, which computes the difference between the maximum and minimum of a series, is invoked once on each column in frame3. The result is a series having the columns of frame3 as its index.

In [ ]:
f = lambda x: x.max() - x.min()
frame3.apply(f)

If you pass axis="columns" to apply, the function will be invoked once per row insteads:

With the applymap method you can use element-wise Python functions:

In [ ]:
format = lambda x: "%.2f" % x # This function rounds each value to the second decimal

In [ ]:
frame3.applymap(format)

### Sorting

Let's create a new DataFrame with random values:

In [ ]:
np.random.seed(42)
frame = pd.DataFrame(np.random.randint(0,50,81).reshape((9, 9)),      # Let's create a random DataFrame
                     index=["a", "c", "i", "d", "f", "g", "b", "e", "h"],
                     columns=["ci", "be", "bu", "bo", "bi", "ce", "ca", "ba", "co"])
frame

We can sort the DataFrame by row with the sort_index method:

In [ ]:
frame.sort_index()

If we need to sort column-wise, we have to pass axis=1 or axis="columns"

In [ ]:
frame.sort_index(axis=1, ascending=False)

If you actually want to sort the values row- or column-wise, you need to use the sort_values method and pass the row or column you want to sort by. You can also pass a list of rows or columns:

In [ ]:
frame.sort_values(by=["ba", "be"]).sort_index(axis=1)

## Summarizing and Computing Descriptive Statistics

The following methods redurn a series containing column reductions (sums or mean etc.):

In [ ]:
frame.sum()

For row wise reduction pass axis="columns":

In [ ]:
frame.sum(axis='columns')

NA values are excluded unless the entire row or column is NA. Do disable this pass skpna=False:

In [ ]:
frame.mean(axis='columns', skipna=False)

To produce multiple summary statistics in one shot, use the describe() method:

In [ ]:
frame.describe()

On non-numeric data, describe produces alternative summary statistics:

In [ ]:
obj = pd.Series(['bli', 'bli', 'bla', 'blu'] * 5)
obj.describe()

### Correlation and Covariance

To demonstrate correlation and covariance we will download a few stock tickers:

In [ ]:
import pandas_datareader.data as web

In [ ]:
all_data = {ticker: web.get_data_yahoo(ticker) for ticker in ["AAPL", "IBM", "MSFT", "GOOG"]}

In [ ]:
price = pd.DataFrame({ticker: data["Adj Close"] for ticker, data in all_data.items()})

In [ ]:
volume = pd.DataFrame({ticker: data["Volume"] for ticker, data in all_data.items()})

Here we compute percent changes of the prices:

In [ ]:
returns = price.pct_change()
returns.tail()

This is how you compute the correlation and the covariance of values in two columns:

In [ ]:
returns['MSFT'].corr(returns['IBM'])

In [ ]:
returns['MSFT'].cov(returns['IBM'])

If you need a full correalation or covariance matrix of your DataFrame, just enter df.corr() or df.cov()

In [ ]:
returns.corr()

### Unique Values and Value Counts

To find out how many unique values you have in a row or column, use the unique() or vlaue_counts() methods:

In [ ]:
pdf4["Origin"].unique()

In [ ]:
pdf4["Origin"].value_counts(sort=True) # The output is not sorted by default

How would you find out wich destination is mentioned most often in our pdf4 DataFrame?

In [ ]:
# It's your turn:


Solution:

In [ ]:
pdf3["Dest"].value_counts(sort=True) #ORD is Chicago's O'Hare airport

## Data Wrangling

### Combining Datasets

Merge and join operations combine datasets along axis 0 (rows). To demonstrate how to use these functions, let us create two new datasets first.

In [ ]:
df1 = pd.DataFrame({"a":[1,1,1,2,2,3,3],
                    "b": np.random.randint(0,99,7)})
df1

In [ ]:
df2 = pd.DataFrame({"a":[1,2,3,4,5],
                    "c":np.random.randint(0,99,5)})
df2

As you can see in the text cell, the merge() function automatically uses overlapping column names as keys, whithout us having to specify one. It is good practice, however to explicitly say on which columng they should merge.

In [ ]:
pd.merge(df1,df2)

You may also notice that values 4 and 5 from column a are missing after the merge. This is because datasets do an inner merge by default, meaning that we have the common set found of both tables in the output. We can however pass how="outer" or "left" or "right" do get the union of the keys or youse the one of the left or right DataFrame.

In [ ]:
pd.merge(df1, df2, on="a", how="outer")

DataFrame has a convenient join instance for mergin by index. It can be used to combinde multiple Dataframes with the same indexes but without overlapping columns.

In [ ]:
df3 = pd.DataFrame({"c":np.random.randint(0,10,7),
                    "d": np.random.randint(0,99,7)})

In [ ]:
df1.join(df3, how="left")

Another form of Dataset combination is called concatenation. You can use that to extend DataFrames along axis 1 (columns) as well as 0 (rows).

In [ ]:
pd.concat([df1,df2,df3])

In [ ]:
pd.concat([df1,df3], axis=1)

### Reshaping DataFrames

The "stack" action rotates columns in the data to the rows and uses hierarchical indexing, "unstack" does the opposite.

In [ ]:
result = frame3.stack()
result

In [ ]:
result.unstack() # This can introduce missing data

The pivot() method returns a reshaped DataFrame organised by given index/column values. Pivoting your data allows you to reshape it in a way that makes it easier to understand or analyse. Often you’ll use a pivot to demonstrate the relationship between two columns that can be difficult show before the pivot.

In [ ]:
stocks = pd.read_csv('https://gist.githubusercontent.com/alexdebrie/b3f40efc3dd7664df5a20f5eee85e854/raw/ee3e6feccba2464cbbc2e185fb17961c53d2a7f5/stocks.csv')
stocks # Let's import a new dataset to demonstrate the pivot method

In [ ]:
# A pivot lets us visualise the change in volume over time for each stock much easier
stocks.pivot(index='symbol', columns='date', values='volume')

### Grouping

Each group operation does a slit-apply-combine action. The data is split into groups based on one or more keys along a particular axis. Then a function is applied to each group, producing a new value. Finally, the results of all those function applications are combidned into a result object.

   <img src="images/Groupby_grafik.png" align="center" width="50%">                             

Let's get our pdf4 flight dataset again and see what the mean departure delay for each is:

In [ ]:
pdf4.head()

In [ ]:
grouped = pdf4["DepDelay"].groupby(pdf4["Origin"])

This code is equivalent with the one above.
In subsequent notebooks you will see more of this syntax.

In [ ]:
grouped = pdf4.groupby("Origin").DepDelay

In [ ]:
grouped.mean()

How would you determine the median arrival delay for each destination airport?

In [ ]:
# It's your turn to code:


Solution:

In [ ]:
grouped_arrival_delay = pdf4["ArrDelay"].groupby(pdf4["Dest"]).median()
grouped_arrival_delay

Now, I'm curious what the maximum median arrival delay is. Let's find out:

In [ ]:
grouped_arrival_delay.max()

Which airport is so notorious?

In [ ]:
grouped_arrival_delay.idxmax()

Hang on, JFK? Are there really flights from one of New York's airports to JFK??? Let's see:

In [ ]:
pdf4[pdf4["Dest"]=="JFK"] # Indeed there are

How would we go about which of the three Origin airports cancelled the most flights? We have to use the pdf3 dataset again and apply the sum() method.

In [ ]:
# Have a go:


Solution:

In [ ]:
# The sort_values just gives you a better overview in large outcomes
pdf3["Cancelled"].groupby(pdf3["Origin"]).sum().sort_values(ascending=False)

## Saving Data

Let's say we would like to save our pdf3 DataFrame as a file. How would we do that? We'll save it in the csv format first. However, we can just as well save it to a json, hdf5 or excel format, to mention just a few.

In [ ]:
pdf3.to_csv("pdf3_nyc_flights.csv")

In [ ]:
pdf3.to_hdf("pdf3_nyc_flights", key="pdf3") # hdf5 files need a key, as they can contain multiple datasets

In [ ]:
pdf5 = pdf3.reset_index() # We need to reset the index as json doesn't allow duplicate indices

In [ ]:
pdf5.to_json("pdf5_nyc_flights")

In [ ]:
pdf3.to_excel("pdf3_nyc_flights") # Oops. Our DataFrame is to large for an excel file...

Please restart the kernel to release the memory:

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(False)